In [ ]:
# Notebook currently does the following:
# 1. Loads data from a json-file
# 2. Balances counts of negative and positive reviews, then saves the data to csv as partitions

# To implement in transformation:
# 1. TF-IDF of text
# 2. Further cleaning: removing unwanted characters and symbols
# 3. Feature engineering: 
#    - length of text
# 4. Transform "unixReviewTime" to datetime format
# 5. Remove punctuation from text features

In [1]:
import pandas as pd
import numpy as np
import pathlib
import re

np.random.seed(0)

In [2]:
CHUNKS_TO_LOAD = 1
CHUNKSIZE = 1000000 # Number of rows to load per chunk, total rows = CHUNKS_TO_LOAD * CHUNKSIZE
DATA_PATH = r"" # Absolute path to data

MAX_WORDS = 150

In [3]:
# Load amazon fashion reviews dataset
reader = pd.read_json(
    DATA_PATH,
    lines=True, 
    compression="gzip",
    chunksize=CHUNKSIZE,
)

features = ["overall", "reviewText", "unixReviewTime", "reviewerID", "reviewerName"]

In [4]:
def remove_punctuation_to_lowercase(text):
    return re.sub(r" +", " ", (re.sub(r"[^a-zA-Z]", " ", text)).lower())

def cut_string_max_words(text):
    return " ".join(text.split()[:MAX_WORDS])

In [5]:
for chunk_n in range(1, CHUNKS_TO_LOAD+1):
    # Remove redundant columns
    df = reader.__next__().loc[:, features]

    # Fix imbalance between scores of 4, 5 and 2, 1.
    value_counts = df["overall"].value_counts()
    count_positive = value_counts[5] + value_counts[4]
    count_neg = value_counts[2] + value_counts[1] 
    rows_to_drop = count_positive - count_neg

    # Equally many negative as positive samples
    df_balanced = df.drop(
        axis=1, 
        index=np.random.choice(
            df.query("overall == 4 or overall == 5").index, 
            rows_to_drop, 
            replace=False
        )
    )   

    # Remove punctuation and convert text to lowercase
    # Shorten reviewTexts longer than MAX_WORDS
    text_features = ["reviewText"]
    
    for feature in text_features:
        
        df_balanced[feature] = df_balanced[feature] \
            .convert_dtypes(convert_string=True) \
            .dropna(axis=0, inplace=False) \
            .apply(remove_punctuation_to_lowercase) \
            .apply(cut_string_max_words) \
            .dropna(axis=0, inplace=False)

    # Remove rows with empty reviewText
    df_balanced = df_balanced.loc[df_balanced["reviewText"] != "", :]

    # Add column that is parsed date of unixReviewTime column
    df_balanced["reviewDate"] = pd.to_datetime(df["unixReviewTime"], unit="s")

    # Extract old filename to construct new name
    path = pathlib.Path(DATA_PATH)

    file_name = path.parts[-1].split(".")[0] + f"_partition_{chunk_n}"
    
    df_balanced.to_csv(f"../../data/transformed/{file_name}.csv")
